In [1]:
import carla
import random
import queue
import numpy as np
import cv2
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import time
import os
import struct
import subprocess
import json
import shutil
from IPython.display import display, clear_output
import torch
from model_wrapper import ModelWrapper
from demo_dataset import DemoDataset
from visual_utils import *
from dash_canvas.utils import array_to_data_url
from display_server import  DisplayServer
import socketio

In [2]:
# Set timestamp and create directory
timestamp = time.strftime("%Y%m%d-%H%M%S")
run_directory = f'runs/run_{timestamp}'
os.makedirs(run_directory, exist_ok=True)

In [3]:
# Function to build the camera projection matrix
def build_projection_matrix(w, h, fov, is_behind_camera=False):
    focal = w / (2.0 * np.tan(fov * np.pi / 360.0))
    K = np.identity(3)
    if is_behind_camera:
        K[0, 0] = K[1, 1] = -focal
    else:
        K[0, 0] = K[1, 1] = focal
    K[0, 2] = w / 2.0
    K[1, 2] = h / 2.0
    return K

In [4]:
# Function to process LiDAR data and save it as a binary file
def save_lidar_data(data):
    points = np.frombuffer(data.raw_data, dtype=np.dtype('f4'))
    points = np.reshape(points, (int(points.shape[0] / 4), 4))
    
    file_name = f'{run_directory}/lidar.bin'
    with open(file_name, 'wb') as f:
        for point in points:
            f.write(struct.pack('ffff', *point))

In [5]:
def convert_cuda_dict_to_numpy(predict_dict):
    numpy_dict = {}
    for key, value in predict_dict.items():
        if value.is_cuda:
            numpy_dict[key] = value.cpu().numpy()
        else:
            numpy_dict[key] = value.numpy()
    return numpy_dict

In [6]:
def generate_ai_predictions(lidar_data):
    points = np.frombuffer(lidar_data.raw_data, dtype=np.dtype('f4'))
    points = np.reshape(points, (int(points.shape[0] / 4), 4))
    points = points.copy()
    points[:, 1] *= -1
    
    # Make a prediction
    with torch.no_grad():
        output = model.forward(points)
    
    return output, points

In [7]:
# Load the model
model = torch.load('models/voxelnext_model_last.pth', weights_only=False)
model.cuda()
model.eval()

/home/polna/Developer/Carla_Client/.venv/lib/python3.10/site-packages/spconv/pytorch/functional.py:47: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  _TORCH_CUSTOM_FWD = amp.custom_fwd(cast_inputs=torch.float16)
/home/polna/Developer/Carla_Client/.venv/lib/python3.10/site-packages/spconv/pytorch/functional.py:97: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  def backward(ctx, grad_output):
/home/polna/Developer/Carla_Client/.venv/lib/python3.10/site-packages/spconv/pytorch/functional.py:163: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  def backward(ctx, grad_output):
/home/polna/Developer/Carla_Client/.venv/lib/python3.10/site-packages/spconv/pytorch/functional.py:243: FutureWarning: `torch.cuda.amp.custom

ModelWrapper(
  (model): VoxelNeXt(
    (vfe): MeanVFE()
    (backbone_3d): VoxelResBackBone8xVoxelNeXt(
      (conv_input): SparseSequential(
        (0): SubMConv3d(4, 32, kernel_size=[3, 3, 3], stride=[1, 1, 1], padding=[1, 1, 1], dilation=[1, 1, 1], output_padding=[0, 0, 0], bias=False, algo=ConvAlgo.MaskImplicitGemm)
        (1): BatchNorm1d(32, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (2): ReLU()
      )
      (conv1): SparseSequential(
        (0): SparseBasicBlock(
          (conv1): SubMConv3d(32, 32, kernel_size=[3, 3, 3], stride=[1, 1, 1], padding=[1, 1, 1], dilation=[1, 1, 1], output_padding=[0, 0, 0], algo=ConvAlgo.MaskImplicitGemm)
          (bn1): BatchNorm1d(32, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (relu): ReLU()
          (conv2): SubMConv3d(32, 32, kernel_size=[3, 3, 3], stride=[1, 1, 1], padding=[1, 1, 1], dilation=[1, 1, 1], output_padding=[0, 0, 0], algo=ConvAlgo.MaskImplicitGemm)
          (bn2)

In [9]:
# Connect to Carla server
client = carla.Client('localhost', 2000)
client.set_timeout(10.0)
world = client.get_world()
tm = client.get_trafficmanager()
tm.set_synchronous_mode(True)

# Load a town (optional)
#town_name = 'Town03'
#world = client.load_world(town_name)

# Set the simulation FPS
desired_fps = 10  # Set your desired FPS here
settings = world.get_settings()
settings.fixed_delta_seconds = 1.0 / desired_fps
settings.synchronous_mode = True  # Enable synchronous mod
world.apply_settings(settings)

6102

In [10]:
#Spawn random vehicles with autopilot
blueprint_library = world.get_blueprint_library()
vehicle_bp = blueprint_library.filter('vehicle.*')

# Get all walker and walker controller blueprints
walker_bp = blueprint_library.filter('walker.pedestrian.*')
controller_bp = blueprint_library.find('controller.ai.walker')

spawn_points = world.get_map().get_spawn_points()

In [11]:
#Spawn random vehicles with autopilot
random.shuffle(spawn_points)
# Limit to 50 cars or available spawn points
num_cars = min(50, len(spawn_points))

vehicles = []

for spawn_point in spawn_points[:num_cars]:
    for _ in range(10):  # Try up to 10 times to spawn the vehicle
        vehicle = world.try_spawn_actor(random.choice(vehicle_bp), spawn_point)
        if vehicle:
            vehicle.set_autopilot(True)
            vehicles.append(vehicle)
            break  # Exit the retry loop if successful

In [12]:
# Spawn pedestrians
num_pedestrians = num_cars * 5  # Five times the number of vehicles
pedestrian_spawn_points = [carla.Transform(location=world.get_random_location_from_navigation())
                           for _ in range(num_pedestrians)
                           if world.get_random_location_from_navigation() is not None]

# Build and apply the batch of commands to spawn the pedestrians
batch = [carla.command.SpawnActor(random.choice(walker_bp), spawn_point)
         for spawn_point in pedestrian_spawn_points]
results = client.apply_batch_sync(batch, True)

# Initialize lists to store pedestrian and controller actors
pedestrians = []
walker_controllers = []

# Process the results of pedestrian spawning
walkers_list = [{"id": result.actor_id} for result in results if not result.error]

# Build and apply the batch of commands to spawn the walker controllers
batch = [carla.command.SpawnActor(controller_bp, carla.Transform(), walker["id"])
         for walker in walkers_list]
results = client.apply_batch_sync(batch, True)

# Process the results of controller spawning and populate the lists
for i, result in enumerate(results):
    if not result.error:
        walkers_list[i]["con"] = result.actor_id
        pedestrians.append(walkers_list[i]["id"])
        walker_controllers.append(walkers_list[i]["con"])

# Retrieve all actors (walkers and controllers) from their IDs
all_id = pedestrians + walker_controllers
all_actors = world.get_actors(all_id)

# Initialize each controller and set a random walk target and speed
for i in range(len(walker_controllers)):
    walker_controller = all_actors.find(walker_controllers[i])
    pedestrian = all_actors.find(pedestrians[i])
    
    walker_controller.start()
    walker_controller.go_to_location(world.get_random_location_from_navigation())
    walker_controller.set_max_speed(1 + random.random())

In [13]:
#Spawn your car with autopilot and LiDAR sensor
ego_vehicle_bp = blueprint_library.find('vehicle.audi.a2')
ego_vehicle_spawn_point = spawn_points[num_cars]

ego_vehicle = world.spawn_actor(ego_vehicle_bp, ego_vehicle_spawn_point)
ego_vehicle.set_autopilot(True)

In [14]:
# Spawn Lidar
lidar_bp = blueprint_library.find('sensor.lidar.ray_cast')
lidar_bp.set_attribute('range', '100')  # Increase range if needed
lidar_bp.set_attribute('rotation_frequency', str(desired_fps))
lidar_bp.set_attribute('channels', '64')  # Increase number of channels for better vertical resolution
lidar_bp.set_attribute('points_per_second', '1000000')  # Increase points per second for denser point cloud
lidar_bp.set_attribute('upper_fov', '10')  # Adjust upper field of view
lidar_bp.set_attribute('lower_fov', '-30')  # Adjust lower field of view
lidar_bp.set_attribute('horizontal_fov', '360')  # Ensure full 360 degree horizontal field of view

lidar_location = carla.Location(0, 0, 2.5)
lidar_rotation = carla.Rotation(0, 0, 0)
lidar_transform = carla.Transform(lidar_location, lidar_rotation)
lidar = world.spawn_actor(lidar_bp, lidar_transform, attach_to=ego_vehicle)

In [15]:
# Spawn camera
camera_bp = blueprint_library.find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '1920')  # 8K resolution width
camera_bp.set_attribute('image_size_y', '1080')  # 8K resolution height

In [16]:
# Define camera transforms for each direction (relative to the car)
camera_front_transform = carla.Transform(carla.Location(x=1.5, y=0.0, z=2.0), carla.Rotation(pitch=0.0, yaw=0.0, roll=0.0))
camera_rear_transform = carla.Transform(carla.Location(x=-1.5, y=0.0, z=2.0), carla.Rotation(pitch=0.0, yaw=180.0, roll=0.0))
camera_left_transform = carla.Transform(carla.Location(x=0.0, y=-0.5, z=2.0), carla.Rotation(pitch=0.0, yaw=-90.0, roll=0.0))
camera_right_transform = carla.Transform(carla.Location(x=0.0, y=0.5, z=2.0), carla.Rotation(pitch=0.0, yaw=90.0, roll=0.0))

In [17]:
# Spawn cameras attached to the ego vehicle
camera_front = world.spawn_actor(camera_bp, camera_front_transform, attach_to=ego_vehicle)
camera_rear = world.spawn_actor(camera_bp, camera_rear_transform, attach_to=ego_vehicle)
camera_left = world.spawn_actor(camera_bp, camera_left_transform, attach_to=ego_vehicle)
camera_right = world.spawn_actor(camera_bp, camera_right_transform, attach_to=ego_vehicle)

In [18]:
# Create a queue to store and retrieve the sensors data
lidar_queue = queue.Queue()
lidar.listen(lidar_queue.put)

camera_front_queue = queue.Queue()
camera_rear_queue = queue.Queue()
camera_left_queue = queue.Queue()
camera_right_queue = queue.Queue()

camera_front.listen(camera_front_queue.put)
camera_rear.listen(camera_rear_queue.put)
camera_left.listen(camera_left_queue.put)
camera_right.listen(camera_right_queue.put)

In [19]:
# Get the attributes from the camera
image_w = camera_bp.get_attribute("image_size_x").as_int()
image_h = camera_bp.get_attribute("image_size_y").as_int()
fov = camera_bp.get_attribute("fov").as_float()

# Calculate the camera projection matrix to project from 3D -> 2D
K = build_projection_matrix(image_w, image_h, fov)

In [20]:
def update_spectator():
    # Get the location and rotation of the ego vehicle
    ego_location = ego_vehicle.get_location()
    ego_rotation = ego_vehicle.get_transform().rotation
    
    # Calculate an offset position behind and above the vehicle
    offset_distance = 5.0  # Distance behind the vehicle
    offset_height = 2.0    # Height above the vehicle
    
    # Convert rotation to radians
    yaw = np.radians(ego_rotation.yaw)
    
    # Calculate the offset position
    offset_x = -offset_distance * np.cos(yaw)
    offset_y = -offset_distance * np.sin(yaw)
    
    # New location for the spectator
    spectator_location = carla.Location(
        x=ego_location.x + offset_x,
        y=ego_location.y + offset_y,
        z=ego_location.z + offset_height
    )
    
    # Set the spectator's transform
    spectator = world.get_spectator()
    spectator.set_transform(carla.Transform(spectator_location, ego_rotation))

In [21]:
def compute_model():
    # Get the latest LiDAR data and generate AI predictions
    lidar_data = lidar_queue.get()
    
    #save_lidar_data(lidar_data)
    predict_dict, points = generate_ai_predictions(lidar_data)
        
    # Make a prediction
    with torch.no_grad():
        predict_dict = model.forward(points)
    
    # Get the latest images from all four camera queues
    image_data_front = camera_front_queue.get()
    image_data_rear = camera_rear_queue.get()
    image_data_left = camera_left_queue.get()
    image_data_right = camera_right_queue.get()
    
    # Process front camera image
    img_front = np.reshape(np.copy(image_data_front.raw_data), (image_data_front.height, image_data_front.width, 4))
    img_front_plt = cv2.cvtColor(img_front, cv2.COLOR_BGRA2RGBA)

    # Process rear camera image
    img_rear = np.reshape(np.copy(image_data_rear.raw_data), (image_data_rear.height, image_data_rear.width, 4))
    img_rear_plt = cv2.cvtColor(img_rear, cv2.COLOR_BGRA2RGBA)

    # Process left camera image
    img_left = np.reshape(np.copy(image_data_left.raw_data), (image_data_left.height, image_data_left.width, 4))
    img_left_plt = cv2.cvtColor(img_left, cv2.COLOR_BGRA2RGBA)

    # Process right camera image
    img_right = np.reshape(np.copy(image_data_right.raw_data), (image_data_right.height, image_data_right.width, 4))
    img_right_plt = cv2.cvtColor(img_right, cv2.COLOR_BGRA2RGBA)
    
    # Convert AI predictions to CPU for further processing
    predict_dict_cpu = convert_cuda_dict_to_numpy(predict_dict)
    
    return points, img_front_plt, img_rear_plt, img_left_plt, img_right_plt, predict_dict_cpu

In [22]:
def simulate_world_plot_data():
    world.tick()
    update_spectator()
    points, img_front, img_rear, img_left, img_right, predict_dict = compute_model()
    visualize_lidar_data_3D(points, predict_dict['pred_boxes'], predict_dict['pred_labels'], predict_dict['pred_scores'], window=True)

In [23]:
def simulate_world(server, websocket):
    world.tick()
    update_spectator()
    points, img_front, img_rear, img_left, img_right, predict_dict = compute_model()
    server.update_data(points, predict_dict['pred_boxes'], predict_dict['pred_labels'], predict_dict['pred_scores'], img_front, img_rear, img_left, img_right)
    websocket.emit('update_data')

In [24]:
# Initialize the Dash application
display_server = DisplayServer()

# Start the server
display_server.start()

# Create a Socket.IO client
sio = socketio.Client()

time.sleep(2)  # Wait for 2 seconds to give the server time to start

# Connect to the server
sio.connect("http://localhost:5000")

(55772) wsgi starting up on http://127.0.0.1:5000
(55772) accepted ('127.0.0.1', 51574)
127.0.0.1 - - [09/Oct/2024 23:22:42] "GET /socket.io/?transport=polling&EIO=4&t=1728508962.6120985 HTTP/1.1" 200 278 0.000322
(55772) accepted ('127.0.0.1', 51582)


In [ ]:
for _ in range(500):
    simulate_world(server=display_server, websocket=sio)
    time.sleep(1.0 / desired_fps)

In [ ]:
simulate_world(server=display_server, websocket=sio)

In [ ]:
simulate_world_plot_data()

# This is to save each frame for optional analysis

In [ ]:
from PIL import Image
# Start loop for simulation
for step in range(1050):  # Total steps of 1050
    # Simulate world ticking and update
    world.tick()
    update_spectator()
    
    # Compute model predictions and images
    points, img_front, img_rear, img_left, img_right, predict_dict = compute_model()
    
    # Only start saving after 50 steps
    if step >= 50:
        frame_index = step - 50  # Frame index starts from 0 after 50 steps
        
        # Save LiDAR visualization
        lidar_save_path = f'gif generator/lidar/frame_{frame_index}.png'
        visualize_lidar_data_3D_3rd_person_view(points, predict_dict['pred_boxes'], predict_dict['pred_labels'], predict_dict['pred_scores'], window=False, save_path=lidar_save_path)
        
        # Save front, rear, left, right images
        Image.fromarray(img_front).save(f'gif generator/img_front/frame_{frame_index}.png')
        Image.fromarray(img_rear).save(f'gif generator/img_rear/frame_{frame_index}.png')
        Image.fromarray(img_left).save(f'gif generator/img_left/frame_{frame_index}.png')
        Image.fromarray(img_right).save(f'gif generator/img_right/frame_{frame_index}.png')
        
        print(f"Saved frame {frame_index}")

print("All frames saved.")

In [ ]:
for _ in range(500):
    world.tick()

lidar_queue.queue.clear()
camera_front_queue.queue.clear()
camera_rear_queue.queue.clear()
camera_left_queue.queue.clear()
camera_right_queue.queue.clear()

In [ ]:
#Cleanup
lidar.stop()
lidar.destroy()

camera_front.stop()
camera_front.destroy()

camera_rear.stop()
camera_rear.destroy()

camera_left.stop()
camera_left.destroy()

camera_right.stop()
camera_right.destroy()

In [ ]:
ego_vehicle.destroy()

In [ ]:
for vehicle in vehicles:
    vehicle.destroy()

In [ ]:
tm.set_synchronous_mode(False)
settings.synchronous_mode = False
settings.fixed_delta_seconds = None
world.apply_settings(settings)

In [ ]:
sio.disconnect()
display_server.stop()